In [5]:
import os
from itertools import zip_longest
from scripts.tools.patcher.src.merge import lv_parse
from collections import OrderedDict

def ld(d):
    for file in os.listdir(d):
        return os.path.join(d, file)
    
    
import difflib
def diff2(a,b):
    if isinstance(a, str):
        a = a.splitlines()
    if isinstance(b, str):
        b = b.splitlines()

    # inserting a blank line here here doesn't impact
    # any real use case, but ensures that the message printed
    # by pytest's assert failure thingy 
    res = [""]
    res.extend(s.strip() for s in difflib.ndiff(a,b) if s[0] in ("+", "-", "?", "!"))
    return res

def diff3(a,b):
    res = diff2(a,b)
    if len(res) == 1 and res[0] == "":  # no diff found
        return ["<Unknown mismatch - processed files are identical. Compare raw text>"]
    return res
    
def rdf(fp):
    with open(fp, 'r') as f:
        return f.read()
rfp = rdf

def loadlv(file, find):
    return lv_parse.lv_parse_string(rfp(find(file)))

def compare(sn):
    l = lvd% sn
    d = rc % sn
    for p in (l,d):
        if not os.path.exists(p):
            print("Error - improper serial number (customer folder not found!)")
            return
    
    def findc(file,dirs=(l,d)):
        for dir in dirs:
            fp = os.path.join(dir, file)
            if os.path.exists(fp):
                return fp
        raise FileNotFoundError(fp)
    
    def findr(file):
        fp = os.path.join(ref, file)
        if not os.path.exists(fp):
            raise FileNotFoundError(fp)
        return fp
    
    t1 = t2 = ""
    def basic(file, cdirs=(l,d), rf=None):
        nonlocal t1, t2
        fc = findc(file, cdirs)
        fr = findr(rf or file)
        t1 = rdf(fc)
        t2 = rdf(fr)
        op = "==" if t1 == t2 else "!="
        print("%-35s: User %s Reference"%(file, op))
        return op == "=="
    
    def b2lv(file):
        return basic(file, (l,))
    
    def b2rc(file):
        return basic(file, (d,))
    
    def b22lv(file):
        return basic2(file, (l,))
    
    def b22rc(file):
        return basic2(file, (d,))
    
    diffs = []
    def basic2(file, cdirs=(l,d), show=True):
        b = basic(file, cdirs)
        if not b and show:
            dl = []
            l1 = t1.splitlines()
            l2 = t2.splitlines()
            for i, (s1, s2) in enumerate(zip_longest(l1, l2, fillvalue=""), 1):
                if s1 != s2:
                    dl.append("Line %d: %r != %r"%(i, s1, s2))
            diffs.append((file, dl))   
        return b
            
    def testlog(file):
        if not basic(file):
            dl = []
            def tabfix(s):
                return s.replace("\t", "    ")
            l1 = list(map(tabfix, t1.splitlines()))
            l2 = list(map(tabfix, t2.splitlines()))
            dl.extend(diff3(l1, l2))            
            if dl:
                diffs.append((file, dl))   
            else:
                # file wasn't sorted properly
                diffs.append((file, ["Improper sort order"]))
            
    def existclv(file):
        fp = os.path.join(l, file)
        if not os.path.exists(fp):
            print("Not found in LabVIEW Data: %s"%file)
        else:
            print("%-35s: Found"%file)
            
    def existcrc(file):
        fp = os.path.join(d, file)
        if not os.path.exists(fp):
            print("Not found in RIO Config: %s"%file)
        else :
            print("%-35s: Found"%file)
            
    def testsv(file):
        if not basic(file, rf="System Variables.sys"):
            lv1 = loadlv(file, findc)
            lv2 = loadlv("System Variables.sys", findr)
            fmt = "%-45s %-6s %-6s != %s"
            dl = [(fmt%("var", "type", "user", "pbs")).replace("!=", "  ")]
            def diff(m, v, a, b):
                if getattr(a, v) != getattr(b, v):
                    dl.append(fmt%(m, v, getattr(a,v), getattr(b,v)))
                    #dl.append("%-20s %s mismatch : %s != %s"%(m, v, getattr(a,v), getattr(b,v)))
            for g1, g2 in zip(lv1, lv2):
                diff("Group", "name", g1, g2)
                diff("Group", "type", g1, g2)
                for v1, v2 in zip(g1, g2):
                    diff("Variable", "name", v1, v2)
                    diff(g1.name+"."+v1.name, "type", v1, v2)
                    diff(g1.name+"."+v1.name, "val", v1, v2)
            diffs.append((file, dl))
            
    def testcal(file):
        if not basic(file):
            lv1 = loadlv(file, findc)
            lv2 = loadlv(file, findr)
            dl = []
            def diff(m, v, a, b):
                if getattr(a, v) != getattr(b, v):
                    dl.append("%-20s %s mismatch : %s != %s"%(m, v, getattr(a,v), getattr(b,v)))
            def diff2(a,b):pass
                
            for rv in lv2:
                cv = lv1.lookup(rv.name)
                diff("Var", "name", rv, cv)
                diff("Var", "type", rv, cv)
                if rv.isiter():
                    for rv2 in rv:
                        cv2 = cv.lookup(rv2.name)
                        diff(rv.name+rv2.name, "name", cv2, rv2)
                        diff(rv.name+rv2.name, "type", cv2, rv2)
                else:
                    diff(rv.name, "name", cv, rv)
                    diff(rv.name, "type", cv, rv)
            if dl:
                diffs.append((file, dl))
            else:
                print("%-35s: File structure verified"%file)
                
    def testrecipes(file):
        if not basic(file, (l,)):
            l1 = t1.splitlines()
            l2 = t2.splitlines()
            crecipe = []
            rrecipe = set()
            dl = []
            for l3 in l1:
                if l3.startswith("Func"):
                    crecipe.append(l3.split(" ",1)[1])
            for l3 in l2:
                if l3.startswith("Func"):
                    rrecipe.add(l3.split(" ", 1)[1])
            for r in crecipe:
                if r not in rrecipe:
                    dl.append("Extra Recipe: %s"%r)
            if dl:
                diffs.append((file, dl))
                    
                
    def findothers():
        xpl = {
            ".alm": ("Alarms On.alm", "Alarms Off.alm"),
            ".sys": ("System Variables.sys",),
            ".log": ("Logging On.log", "Logging Off.log")
        }
        xprc = {
            ".alm": ("Alarms.alm",),
            ".sys": ("System Variables.sys",),
            ".log": ("Logger Settings.log",)
        }
        for xp, dir in zip((xpl, xprc), (l, d)):
            for file in os.listdir(dir):
                ext = os.path.splitext(file)[1]
                if ext not in xp: continue
                if file not in xp[ext]:
                    print("Found extra file: %s"%("\\".join((dir.split("\\")[-1], file))))
    
    # meat 
    b22lv("Alarms Off.alm")
    b22lv("Alarms On.alm")
    #b22rc("Alarms.alm")
    
    testrecipes("Bioreactor Recipes.cfg")
    b22lv("Email Alerts Settings.xml")
    
    testlog("Logging Off.log")
    testlog("Logging On.log")
    testlog("Logger Settings.log")
    
    b22lv("ReportSettings.cfg")
    
    for file in os.listdir(l):
        if file.endswith(".sys"):
            testsv(file)
    #testsv("System Variables.sys")
    
    
    existclv("Email Password.cfgx")
    existclv("Network Drives.cfgx")
    existclv("PBS Users.conf")
    
    #testcal("Cal Factors.cfg")
    
    #b22rc("Bioreactor Configuration.txt")
    #b22rc("pH Control.txt")
    
    #existcrc("Bag Info.cfg")
    #existcrc("Global Variables I.conf")
    #existcrc("Global Variables II.conf")
    #existcrc("Global Variables III.conf")
    
    #findothers()
    

    def showdiff():
        for fp, lines in diffs:
            print()
            print("%s:"%fp)
            for line in lines:
                print(" "*3, line)
    showdiff()
            
    def checklogger():
        l1 = rdf(findc("Logging On.log")).splitlines()
        l2 = rdf(findr("Logging On.log")).splitlines()
        def toset(l):
            s = set()
            for line in l:
                if line:
                    s.add(line.split("\t", 1)[0])
            return s
        def tabfix(s):
            return s.replace("\t", "    ")
        s1 = toset(map(tabfix, l1))
        s2 = toset(map(tabfix, l2))
        print("Extra in customer: ", s1 - s2)
        print("Extra in reference: ", s2 - s1)
    #checklogger()

In [3]:
ref = 'D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 03'
lvd = 'D:\\Customer_Backups\\%s\\LabVIEW_Data'
rc  = 'D:\\Customer_Backups\\%s\\Rio_Config'

In [4]:
import contextlib, sys
@contextlib.contextmanager
def redirect_stdout(fp):
    stdout = sys.stdout
    with open(fp, 'w') as f:
        sys.stdout = f
        try:
            yield
        finally:
            sys.stdout=stdout

In [94]:
class DumbDumb(Exception): pass
sns = ["PBS 3 CDI %d"%i for i in range(2,7)]
ref = 'D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 03'
sns.append("PBS 15 CDI 1")
for sn in sns:
    with redirect_stdout(sn+" simple report.txt"):
        if sn.split()[1] == "3":
            ref = 'D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 03'
        elif sn.split()[1] == "15":
            ref = 'D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 15'
        else:
            raise DumbDumb()
        compare(sn)

In [20]:
ref = 'D:\\auto_hd_install\\default configs\\IM00226 Rev C\\Mag 15'
cdisn = "PBS 3 CDI 5"
compare(cdisn)

Alarms Off.alm                     : User == Reference
Alarms On.alm                      : User == Reference
Alarms.alm                         : User == Reference
Bioreactor Recipes.cfg             : User == Reference
Email Alerts Settings.xml          : User != Reference
Logging Off.log                    : User != Reference
Logging On.log                     : User != Reference
ReportSettings.cfg                 : User == Reference
System Variables.sys               : User != Reference
Email Password.cfgx                : Found
Network Drives.cfgx                : Found
PBS Users.conf                     : Found
Cal Factors.cfg                    : User != Reference
Cal Factors.cfg                    : File structure verified
Bioreactor Configuration.txt       : User == Reference
pH Control.txt                     : User == Reference
Bag Info.cfg                       : Found
Global Variables I.conf            : Found
Global Variables II.conf           : Found
Global Variables III.

In [22]:
cdisn = "PBS 3 CDI 6"
compare(cdisn)

Alarms Off.alm                     : User == Reference
Alarms On.alm                      : User == Reference
Alarms.alm                         : User == Reference
Bioreactor Recipes.cfg             : User == Reference
Email Alerts Settings.xml          : User != Reference
Logging Off.log                    : User != Reference
Logging On.log                     : User != Reference
ReportSettings.cfg                 : User == Reference
System Variables.sys               : User != Reference
Email Password.cfgx                : Found
Network Drives.cfgx                : Found
PBS Users.conf                     : Found
Cal Factors.cfg                    : User != Reference
Cal Factors.cfg                    : File structure verified
Bioreactor Configuration.txt       : User == Reference
pH Control.txt                     : User == Reference
Bag Info.cfg                       : Found
Global Variables I.conf            : Found
Global Variables II.conf           : Found
Global Variables III.

In [ ]:
scrum1sn = "000317S1902 180207"
#compare(scrum1sn)

In [ ]:
sn = "000317S1903_check 180207"
compare(sn)

In [ ]:
sn = "000318J0302_check 180214"
compare(sn)

In [6]:
sn = "000314P2201_backup 180326"
compare(sn)

Alarms Off.alm                     : User == Reference
Alarms On.alm                      : User == Reference
Bioreactor Recipes.cfg             : User != Reference
Email Alerts Settings.xml          : User != Reference
Logging Off.log                    : User == Reference
Logging On.log                     : User == Reference
Logger Settings.log                : User != Reference
ReportSettings.cfg                 : User == Reference
OQ-Alarms Verification.sys         : User != Reference
sv backup.sys                      : User != Reference
System Variables.sys               : User != Reference
Email Password.cfgx                : Found
Network Drives.cfgx                : Found
PBS Users.conf                     : Found

Bioreactor Recipes.cfg:
    Extra Recipe: intelliaheat
    Extra Recipe: SampleTest
    Extra Recipe: OQ-Recipe1
    Extra Recipe: OQ-PowerLoss

Email Alerts Settings.xml:
    Line 2: 'Subject,PBS 3 Mag Eng 1' != 'Subject,PBS'
    Line 5: 'Return Address,pbs@pbsbio

    + MFCN2OutRaw(V).Period(iter)    0.1    FALSE
    + MFCO2FlowFeedback(LPM)    0.025    TRUE
    + MFCO2MeasRaw(V)    0.1    FALSE
    + MFCO2Min(LPM)    0.001    FALSE
    + MFCO2Off(V)    0.001    FALSE
    + MFCO2OutRaw.AO(V)    0.1    FALSE
    + MFCO2OutRaw.On(iter)    0.1    FALSE
    + MFCO2OutRaw.Period(iter)    0.1    FALSE
    + MFCOnTime(s)    0.001    FALSE
    + O2 Min Volume (L)    0.001    FALSE
    + InterlockAgMotor    0.5    FALSE
    + InterlockAll    0.5    FALSE
    + InterlockDoor    0.5    FALSE
    + InterlockDoorPressureMax(psi)    0.001    FALSE
    + InterlockGasFlow    0.5    FALSE
    + InterlockHeater    0.5    FALSE
    + InterlockPressureMax(psi)    0.001    FALSE
    + InterlockPumps    0.5    FALSE
    + InterlockTempMax(C)    0.001    FALSE
    + LEDWhiteLEDOn    0.5    FALSE
    + LevelCalCluster.Bottom Gap (cm)    0.001    FALSE
    + LevelCalCluster.Cm/psi    0.001    FALSE
    + LevelCalCluster.Depth    0.001    FALSE
    + LevelCalCluster.Leve